In [1]:
import pandas as pd
import json
import os

In [3]:
submission_path = "/scratch/midway3/yingrong/MEME_data/meme_data/filtered_submission.json"
with open(submission_path, "r") as json_file:
    submission_dic = json.load(json_file)

In [4]:
comments_path = "/scratch/midway3/yingrong/MEME_data/meme_data/filtered_comments.json"
with open(comments_path, "r") as f:
    comments_dic = json.load(f)

In [14]:
combined_data = {}

for i in range(10):
    file = f"extracted_text_{i}.json"    #the datesets of text extracted from meme images
    with open(file, 'r') as f:
        data = json.load(f)
        print(f"The length of file {i} is {len(data)}")
        combined_data.update(data)
print(f"Combination finished! The length of total data is {len(combined_data)}")

The length of file 0 is 5000
The length of file 1 is 15000
The length of file 2 is 10000
The length of file 3 is 10000
The length of file 4 is 10000
The length of file 5 is 10000
The length of file 6 is 10000
The length of file 7 is 10000
The length of file 8 is 10000
The length of file 9 is 12231
Combination finished! The length of total data is 102231


In [6]:
templated_path = "/home/yingrong/meme_project/time_series/templated_meme2.json"
with open(templated_path, "r") as json_file:
    data_templated = json.load(json_file)

In [7]:
all_paths = []

# Iterate through each cluster and extract paths
for cluster in data_templated:
    for key, paths in cluster.items():
        if len(paths) > 100:
            all_paths.extend(paths)

print(f"The length of all_paths is {len(all_paths)}")

The length of all_paths is 102096


In [9]:
new_dic = {}
for cluster in data_templated:
    for key, paths in cluster.items():
        if len(paths) > 100:
            for path in paths:
                new_dic[path] = key

In [17]:
%%time

all_lst = []
for file_path in all_paths:
    new_lst = []
    meme_id = os.path.splitext(os.path.basename(file_path))[0]
    new_lst.append(meme_id)
    new_lst.append(new_dic[file_path])  # cluster name
    post_info = submission_dic[meme_id]
    new_lst.append(post_info["created_utc"])
    new_lst.append(post_info["title"])
    new_lst.append(post_info["selftext"])
    new_lst.append(post_info["author"])
    new_lst.append(post_info["num_comments"])
    new_lst.append(post_info["score"])
    new_lst.append(combined_data[meme_id])
    comments_lst = comments_dic[meme_id]  # extracted_text from image
    comment_text = ""
    for comment in comments_lst:
        for k, v in comment.items():
            comment_text = comment_text + v["body"] + "\n"
    new_lst.append(comment_text)
    all_lst.append(new_lst)
    
columns = ["meme_id","cluster", "created_utc", "title","selftext","author","num_comments","score","extracted_text", "comments_text"]
df = pd.DataFrame(all_lst, columns=columns)
df.head(10)

CPU times: user 3.71 s, sys: 1.06 s, total: 4.77 s
Wall time: 4.81 s


,meme_id,cluster,created_utc,title,selftext,author,num_comments,score,extracted_text,comments_text
0,cawlo8,cluster_4,1562650591,"I have an MIDI keyboard, just no plug for it ):",,YTMusicXD,0,3,When you want to easily make midi; limited pow...,
1,d72m1n,cluster_4,1569020677,R.I.P,,masnargom,0,95,When Imina room full of antivax children and f...,
2,bo6k2s,cluster_4,1557768971,It's like a life hack,,emanuelegay,2,4,Connect one Power Bank to another one Unlimite...,This is just plug in a power strip to itself\n...
3,cr40bc,cluster_4,1565947892,I Finally Have Unlimited Power,,Zaxoosh,0,64,When you finally get enough karma to post on r...,
4,apoel0,cluster_4,1549939436,Crrevt,,memeboiiii7059,0,7,whwn yoy make a meme wirh one hand Unlimited p...,
5,alz0jk,cluster_4,1549000796,Uno Block Card,,Your_Lost_Wig,0,20,when you plug a charger in a charger Unlimited...,
6,bng0ts,cluster_4,1557603290,When i wake up,,tiesking0_0,0,1,Her : you wanna come over Me : Unlimited powerl,
7,b1w0r6,cluster_4,1552763377,half of this sub,,ZebraGames69,0,12,when you dont have enough karma to post to r/d...,
8,bnejce,cluster_4,1557595328,When i wake up,,tiesking0_0,0,1,Her : you wanna come over Me : Unlimited powerl,
9,b331qr,cluster_4,1553030364,First post,,Datmeme247,3,10,When you get a meme maker app Unlimited powerl,When you make a meme making app:\n*Thanos show...


In [19]:
df['created_date'] = pd.to_datetime(df['created_utc'], unit='s').dt.strftime('%Y-%m-%d')

In [20]:
df.head()

,meme_id,cluster,created_utc,title,selftext,author,num_comments,score,extracted_text,comments_text,created_date
0,cawlo8,cluster_4,1562650591,"I have an MIDI keyboard, just no plug for it ):",,YTMusicXD,0,3,When you want to easily make midi; limited pow...,,2019-07-09
1,d72m1n,cluster_4,1569020677,R.I.P,,masnargom,0,95,When Imina room full of antivax children and f...,,2019-09-20
2,bo6k2s,cluster_4,1557768971,It's like a life hack,,emanuelegay,2,4,Connect one Power Bank to another one Unlimite...,This is just plug in a power strip to itself\n...,2019-05-13
3,cr40bc,cluster_4,1565947892,I Finally Have Unlimited Power,,Zaxoosh,0,64,When you finally get enough karma to post on r...,,2019-08-16
4,apoel0,cluster_4,1549939436,Crrevt,,memeboiiii7059,0,7,whwn yoy make a meme wirh one hand Unlimited p...,,2019-02-12


In [21]:
len(df)

102096

In [22]:
df = df.drop_duplicates(["title", "author","extracted_text","created_date"])

In [23]:
len(df)

100431

In [24]:
df.nunique()

meme_id           100431
cluster              460
created_utc       100368
title              78658
selftext              49
author             50583
num_comments         504
score               2552
extracted_text     94971
comments_text      51775
created_date        2549
dtype: int64

In [27]:
df.to_csv("/scratch/midway3/yingrong/MEME_data/meme_data/meme_table.csv", index=False)

In [26]:
df["selftext"].value_counts()

                                                                                                                                                                                                                                                                                                                                                                    99948
[deleted]                                                                                                                                                                                                                                                                                                                                                             422
[removed]                                                                                                                                                                                                                                                                           